In [1]:
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(888)  # for reproducibility

C:\Python\Anaconda3_5.2.0\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Predicting animal type based on various features
# xy = np.loadtxt('data/train5.csv', delimiter=',', dtype=np.float32)
xy = np.loadtxt('data/train_householder2.csv', delimiter=',', dtype=np.float32)

In [4]:
# ML202-Softmax_Classifier 참고!!

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)
print('\nx_data :\n', x_data)
print('\ny_data :\n', y_data)

nb_classes = 4  # 0~3

X = tf.placeholder(tf.float32, [None, 111])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 3

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("\none_hot", Y_one_hot)

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("\nreshape", Y_one_hot)

W = tf.Variable(tf.random_normal([111, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(2973, 111) (2973, 1)

x_data :
 [[0. 3. 0. ... 0. 1. 0.]
 [0. 4. 0. ... 0. 1. 0.]
 [0. 8. 0. ... 0. 1. 0.]
 ...
 [0. 5. 0. ... 1. 0. 1.]
 [0. 3. 0. ... 1. 0. 1.]
 [0. 6. 0. ... 1. 0. 1.]]

y_data :
 [[3.]
 [3.]
 [3.]
 ...
 [3.]
 [1.]
 [1.]]

one_hot Tensor("one_hot:0", shape=(?, 1, 4), dtype=float32)

reshape Tensor("Reshape:0", shape=(?, 4), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [5]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 1000 == 0 or step < 10:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: 10.432, 	 Acc: 26.71%
Step:     1, 	 Loss: 9.521, 	 Acc: 29.26%
Step:     2, 	 Loss: 8.735, 	 Acc: 32.19%
Step:     3, 	 Loss: 8.057, 	 Acc: 34.28%
Step:     4, 	 Loss: 7.470, 	 Acc: 36.23%
Step:     5, 	 Loss: 6.961, 	 Acc: 37.84%
Step:     6, 	 Loss: 6.521, 	 Acc: 39.29%
Step:     7, 	 Loss: 6.142, 	 Acc: 41.20%
Step:     8, 	 Loss: 5.818, 	 Acc: 42.45%
Step:     9, 	 Loss: 5.541, 	 Acc: 44.06%
Step:  1000, 	 Loss: 2.002, 	 Acc: 57.42%
Step:  2000, 	 Loss: 1.585, 	 Acc: 59.87%
Step:  3000, 	 Loss: 1.347, 	 Acc: 61.45%
Step:  4000, 	 Loss: 1.197, 	 Acc: 62.50%
Step:  5000, 	 Loss: 1.097, 	 Acc: 63.67%
Step:  6000, 	 Loss: 1.028, 	 Acc: 65.02%
Step:  7000, 	 Loss: 0.979, 	 Acc: 65.96%
Step:  8000, 	 Loss: 0.942, 	 Acc: 66.06%
Step:  9000, 	 Loss: 0.915, 	 Acc: 67.14%
Step: 10000, 	 Loss: 0.894, 	 Acc: 67.44%


In [6]:
# # Let's see if we can predict
# pred = sess.run(prediction, feed_dict={X: x_data})

# # y_data: (N,1) = flatten => (N, ) matches pred.shape
# for p, y in zip(pred, y_data.flatten()):
#     print("[{}]  Prediction : {},  True Y : {}".format(p == int(y), p, int(y)))
#     # print("[{}]  Prediction : {},  True Y : {}, y_data : {}".format(p == int(y), p, int(y), y_data))

In [8]:
# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(201)):
        cost_val, W_val, _ = sess.run(
            [cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        if step%20 == 0:
            print("\n Step : {} \n Cost : {} \n Weight : \n{}".format(step, cost_val, W_val))

    print("-"*60)
    # predict
    print("Prediction \t:", sess.run(prediction, feed_dict={X: x_data}))
    # Calculate the accuracy
    print("Accuracy \t: ", sess.run(accuracy, feed_dict={X: x_data, Y: y_data}))


 Step : 0 
 Cost : 11.483365058898926 
 Weight : 
[[-0.39064652  0.49924093  0.360921    0.15001689]
 [ 0.04784523 -1.3655696  -0.22568236  0.8898793 ]
 [-1.2248679  -1.4991615   0.7397786   1.014317  ]
 [ 1.3415849  -0.6845606  -0.7505039  -0.4623365 ]
 [-0.48526406  0.6822929  -1.8286966  -0.6045906 ]
 [ 0.5526507   1.2683839  -0.9745161   0.26286617]
 [-0.1914331   1.338514   -0.647488   -0.09388788]
 [ 0.68202865  0.20353287  0.49015576 -0.8246363 ]
 [ 1.1253166  -0.41138956  1.8372089  -1.0488483 ]
 [ 0.51036245  0.7181332   0.13507877 -0.89009255]
 [ 1.397099    0.05104917  0.81159097  0.5212088 ]
 [-1.3312322  -0.36758462 -1.6028177   1.5522292 ]
 [ 0.81540513 -0.6949389   1.3605374   0.15844099]
 [-0.19280791  0.90576583 -0.41609156  0.5345095 ]
 [-0.7255851  -0.81891936  1.375327   -1.3929769 ]
 [ 0.19029741  0.46511272  0.10488108 -0.15143678]
 [-1.4170218   0.13998252 -1.3812318  -1.5485222 ]
 [ 0.18985261 -0.91651654  0.70847076 -1.763563  ]
 [-0.5720468   0.9596215   0.49

 [ 0.02044707 -0.28076     1.3229711  -1.4082351 ]]

 Step : 40 
 Cost : 3.8227479457855225 
 Weight : 
[[-3.88412058e-01  5.00212312e-01  3.55652571e-01  1.52079597e-01]
 [ 1.32257283e-01 -1.38004410e+00 -4.92551923e-01  1.08681118e+00]
 [-1.22380078e+00 -1.49815381e+00  7.36495078e-01  1.01552677e+00]
 [ 1.36105502e+00 -6.88833058e-01 -8.09727728e-01 -4.18310195e-01]
 [-4.68238741e-01  6.76027775e-01 -1.88145065e+00 -5.62596560e-01]
 [ 5.53925693e-01  1.25034022e+00 -9.76496160e-01  2.81615138e-01]
 [-1.89112186e-01  1.31035662e+00 -6.49301410e-01 -6.62376359e-02]
 [ 6.93770409e-01  1.92107514e-01  4.64212447e-01 -7.99009562e-01]
 [ 1.14181280e+00 -4.30392891e-01  1.70776212e+00 -9.16894853e-01]
 [ 5.38600802e-01  6.87704504e-01 -2.03111544e-02 -7.32512534e-01]
 [ 1.40988612e+00  5.46047315e-02  7.81056106e-01  5.35400689e-01]
 [-1.29972458e+00 -3.58999670e-01 -1.70989108e+00  1.61921048e+00]
 [ 8.59700143e-01 -6.82798564e-01  1.22292888e+00  2.39614308e-01]
 [-1.68278709e-01  8.9789

 [ 0.02030193 -0.2705314   1.3093914  -1.4047385 ]]

 Step : 80 
 Cost : 3.5837149620056152 
 Weight : 
[[-3.8683504e-01  5.0039238e-01  3.5205311e-01  1.5392183e-01]
 [ 1.4691523e-01 -1.3495016e+00 -4.7371200e-01  1.0227709e+00]
 [-1.2230433e+00 -1.4975387e+00  7.3392540e-01  1.0167238e+00]
 [ 1.3643744e+00 -6.8378264e-01 -8.0850679e-01 -4.2790097e-01]
 [-4.6620479e-01  6.7883074e-01 -1.8764627e+00 -5.7242107e-01]
 [ 5.5421185e-01  1.2386997e+00 -9.7273868e-01  2.8921187e-01]
 [-1.8805495e-01  1.2919675e+00 -6.4442277e-01 -5.3783622e-02]
 [ 6.9614571e-01  1.8112525e-01  4.6027043e-01 -7.8646094e-01]
 [ 1.1328801e+00 -4.4301757e-01  1.6732495e+00 -8.6082506e-01]
 [ 5.3204387e-01  6.6409725e-01 -5.8765702e-02 -6.6389382e-01]
 [ 1.4126307e+00  5.9727695e-02  7.7548677e-01  5.3310221e-01]
 [-1.2899998e+00 -3.4257373e-01 -1.7170243e+00  1.6001923e+00]
 [ 8.7216979e-01 -6.6124976e-01  1.2102263e+00  2.1829818e-01]
 [-1.6315852e-01  8.9203656e-01 -4.8208088e-01  5.8457834e-01]
 [-6.7678827e-

 [ 1.48161016e-02 -2.60305971e-01  1.30310762e+00 -1.40319431e+00]]

 Step : 120 
 Cost : 3.4008312225341797 
 Weight : 
[[-0.38559428  0.5004766   0.3488964   0.15575346]
 [ 0.14658193 -1.319095   -0.43523872  0.9542244 ]
 [-1.2224753  -1.4969994   0.731629    1.0179131 ]
 [ 1.3641055  -0.67900884 -0.8032775  -0.4376351 ]
 [-0.46738118  0.68147355 -1.8678577  -0.5824921 ]
 [ 0.55432516  1.2273175  -0.9687704   0.29651257]
 [-0.18724135  1.2738991  -0.6393014  -0.04165019]
 [ 0.6962789   0.17016856  0.4589034  -0.7742704 ]
 [ 1.1175634  -0.45521587  1.6477746  -0.8078349 ]
 [ 0.51686025  0.64094245 -0.08560797 -0.5987132 ]
 [ 1.4129134   0.0649951   0.7723559   0.5306828 ]
 [-1.2850984  -0.3257445  -1.7156062   1.5770435 ]
 [ 0.87735385 -0.6391531   1.2085139   0.19272994]
 [-0.16274239  0.8863475  -0.4865789   0.59434944]
 [-0.6872038  -0.820906    1.0731043  -1.1271486 ]
 [ 0.25874415  0.44370806 -0.26782924  0.1742316 ]
 [-1.3489425   0.11663465 -1.7541714  -1.2203149 ]
 [ 0.2637954

 [ 0.00863346 -0.2502856   1.2970562  -1.4009811 ]]

 Step : 160 
 Cost : 3.238579273223877 
 Weight : 
[[-0.38444436  0.5005294   0.34586644  0.1575807 ]
 [ 0.14565903 -1.2894223  -0.39646152  0.8866975 ]
 [-1.221954   -1.4964902   0.7294171   1.0190942 ]
 [ 1.3634825  -0.6746389  -0.79828495 -0.44637492]
 [-0.46879575  0.6838264  -1.8594971  -0.591791  ]
 [ 0.5544491   1.2160293  -0.9647727   0.30367944]
 [-0.18642065  1.2559448  -0.6341455  -0.02967244]
 [ 0.6961551   0.15914175  0.4580525  -0.762269  ]
 [ 1.1020429  -0.4666156   1.6234635  -0.75660455]
 [ 0.50121605  0.6185161  -0.11076927 -0.5354813 ]
 [ 1.4126757   0.07028838  0.76947576  0.52850735]
 [-1.2800081  -0.30850497 -1.7127173   1.5518248 ]
 [ 0.8822065  -0.6166203   1.2085226   0.16533576]
 [-0.1631038   0.8806141  -0.49031004  0.60417557]
 [-0.697634   -0.81506634  1.0516827  -1.1011368 ]
 [ 0.24795242  0.44381422 -0.2929818   0.21006957]
 [-1.3598541   0.11607223 -1.7794985  -1.183514  ]
 [ 0.2538155  -0.9295562   0.

 [ 2.26838887e-03 -2.40450144e-01  1.29058087e+00 -1.39797640e+00]]

 Step : 200 
 Cost : 3.0950629711151123 
 Weight : 
[[-3.83359641e-01  5.00537515e-01  3.42957169e-01  1.59397170e-01]
 [ 1.44960761e-01 -1.26041460e+00 -3.59227389e-01  8.21154118e-01]
 [-1.22146356e+00 -1.49602795e+00  7.27293491e-01  1.02026510e+00]
 [ 1.36272633e+00 -6.70621455e-01 -7.93900549e-01 -4.54020739e-01]
 [-4.70244557e-01  6.85914814e-01 -1.85167921e+00 -6.00248337e-01]
 [ 5.54588616e-01  1.20482230e+00 -9.60774183e-01  3.10748369e-01]
 [-1.85575262e-01  1.23811316e+00 -6.28985405e-01 -1.78460069e-02]
 [ 6.95957720e-01  1.47976771e-01  4.57633227e-01 -7.50487328e-01]
 [ 1.08684397e+00 -4.77132082e-01  1.59976578e+00 -7.07191467e-01]
 [ 4.85819697e-01  5.96834779e-01 -1.34886324e-01 -4.74286407e-01]
 [ 1.41204858e+00  7.55136460e-02  7.66756773e-01  5.26628256e-01]
 [-1.27441335e+00 -2.91026086e-01 -1.70895243e+00  1.52498603e+00]
 [ 8.87173951e-01 -5.93916118e-01  1.20956826e+00  1.36618242e-01]
 [-1.639

In [ ]:
# ML511-Softmax_for_MNIST 참고!!

In [33]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

xy = np.loadtxt('data/train_householder3.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-4]
y_data = xy[:, -4:]

tf.set_random_seed(999)  # reproducibility


# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, 111])
Y = tf.placeholder(tf.float32, [None, 4])

# weights & bias for nn layers
W = tf.Variable(tf.random_normal([111, 4]))
b = tf.Variable(tf.random_normal([4]))

hypothesis = tf.matmul(X, W) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [35]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 1000 == 0 or step < 10:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: 12.137, 	 Acc: 14.06%
Step:     1, 	 Loss: 12.034, 	 Acc: 14.19%
Step:     2, 	 Loss: 11.931, 	 Acc: 14.46%
Step:     3, 	 Loss: 11.829, 	 Acc: 14.80%
Step:     4, 	 Loss: 11.727, 	 Acc: 15.07%
Step:     5, 	 Loss: 11.625, 	 Acc: 15.17%
Step:     6, 	 Loss: 11.524, 	 Acc: 15.44%
Step:     7, 	 Loss: 11.423, 	 Acc: 15.54%
Step:     8, 	 Loss: 11.323, 	 Acc: 15.71%
Step:     9, 	 Loss: 11.224, 	 Acc: 16.01%
Step:  1000, 	 Loss: 1.954, 	 Acc: 58.69%
Step:  2000, 	 Loss: 1.091, 	 Acc: 64.48%
Step:  3000, 	 Loss: 0.853, 	 Acc: 68.68%
Step:  4000, 	 Loss: 0.792, 	 Acc: 69.42%
Step:  5000, 	 Loss: 0.773, 	 Acc: 70.40%
Step:  6000, 	 Loss: 0.767, 	 Acc: 70.47%
Step:  7000, 	 Loss: 0.765, 	 Acc: 70.47%
Step:  8000, 	 Loss: 0.765, 	 Acc: 70.57%
Step:  9000, 	 Loss: 0.764, 	 Acc: 70.50%
Step: 10000, 	 Loss: 0.764, 	 Acc: 70.57%


In [ ]:
# 샤비어

In [37]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [38]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

xy = np.loadtxt('data/train_householder3.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-4]
y_data = xy[:, -4:]

tf.set_random_seed(999)  # reproducibility


# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, 111])
Y = tf.placeholder(tf.float32, [None, 4])

W1 = tf.get_variable("W1", shape=[111, 64],
                     initializer=xavier_init(111, 64))
b1 = tf.Variable(tf.random_normal([64]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[64, 64],
                     initializer=xavier_init(64, 64))
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[64, 4],
                     initializer=xavier_init(64, 4))
b3 = tf.Variable(tf.random_normal([4]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [39]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(10001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 1000 == 0 or step < 10:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: 1.103, 	 Acc: 64.11%
Step:     1, 	 Loss: 1.054, 	 Acc: 65.62%
Step:     2, 	 Loss: 1.028, 	 Acc: 65.69%
Step:     3, 	 Loss: 1.003, 	 Acc: 65.69%
Step:     4, 	 Loss: 0.978, 	 Acc: 65.69%
Step:     5, 	 Loss: 0.962, 	 Acc: 65.69%
Step:     6, 	 Loss: 0.952, 	 Acc: 65.62%
Step:     7, 	 Loss: 0.942, 	 Acc: 65.79%
Step:     8, 	 Loss: 0.931, 	 Acc: 65.96%
Step:     9, 	 Loss: 0.917, 	 Acc: 66.26%
Step:  1000, 	 Loss: 0.026, 	 Acc: 99.93%
Step:  2000, 	 Loss: 0.002, 	 Acc: 100.00%
Step:  3000, 	 Loss: 0.001, 	 Acc: 100.00%
Step:  4000, 	 Loss: 0.000, 	 Acc: 100.00%
Step:  5000, 	 Loss: 0.000, 	 Acc: 100.00%
Step:  6000, 	 Loss: 0.000, 	 Acc: 100.00%
Step:  7000, 	 Loss: 0.000, 	 Acc: 100.00%
Step:  8000, 	 Loss: 0.000, 	 Acc: 100.00%
Step:  9000, 	 Loss: 0.000, 	 Acc: 100.00%
Step: 10000, 	 Loss: 0.000, 	 Acc: 100.00%
